In [1]:
# import cvxpy as cp
from cvxpy import *
import pandas as pd
import numpy as np
import math
np.set_printoptions(suppress=True)
np.seterr(invalid='ignore')


{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [2]:
def calculate_portfolio_risk(POS, FACTOR_EXP, SIGMA):
    t1 = np.matmul(np.matrix(POS), np.matrix(FACTOR_EXP))
    t1 = np.matmul(t1, np.matrix(SIGMA))
    t1 = np.matmul(t1, np.matrix(FACTOR_EXP).T)
    t1 = np.matmul(t1, np.matrix(POS).T)
    return math.sqrt(t1)


def calculate_portfolio_risk_factor(POS, FACTOR_EXP, SIGMA, SPECIFIC):
    t1 = np.matmul(np.matrix(POS), np.matrix(FACTOR_EXP))
    t1 = np.matmul(t1, np.matrix(SIGMA))
    t1 = np.matmul(t1, np.matrix(FACTOR_EXP).T)
    t1 = np.matmul(t1, np.matrix(POS).T)
    system_risk = t1
    
    t2 = np.matmul(np.matrix(POS), np.matrix(SPECIFIC))
    t2 = np.matmul(t2, np.matrix(POS).T)
    total_risk = math.sqrt(t1 + t2)
    return total_risk



In [77]:
# *************************************
# Optimization - with Trading Cost 
# n stocks
# alp as ALPHA
# n - alp as HEDGE
# factor of 3 x 3
# solve with w_new, w_old with $ positions
np.random.seed(3)
n = 5
numFactor = 3
dataseries = {}
rowlist = []
# Generate 500 price return series from 10 names
for i, v in enumerate(range(numFactor)):
    dataseries[v] = ((1 - np.random.lognormal(0, 0.03, numFactor)))
    
for ke in dataseries.keys():
    rowlist.append(dataseries[ke])

# Generate Timeseries into pd
timeseries = pd.DataFrame(rowlist)
vcv = timeseries.cov()
factor_exp = pd.DataFrame(np.random.randn(n, numFactor))
specific_risk = pd.DataFrame(np.diag(np.random.uniform(0, 0.0020, n)))

# set position vector
w_old = [1000000., 1000000., 2000000., 0., 0.]

# position constraints
w_lb = [999999.9, 999999.9, 1999999., -5000000., -5000000.]
w_ub = [1000000.1, 1000000.1, 2000000.1, 5000000., 5000000.]

a1 = np.matmul(np.matrix(w_old), np.matrix(factor_exp))
print("Current Portfolio Factor Exposure:" + str(a1 / 1000000))
# # factor constraints
# Current exp: -2.16, -4.11, -2.25
#l = [x * 2 for x in l]
f_lb = [1000000 * x for x in [-1., -1., -1.]]
f_ub = [1000000 * x for x in [ 5.,  5.,  5.]]


# # Set up the quadratic optimization problem
w = Variable(n)
f = np.array(factor_exp).T * w 

# Gross Constraints
# sum(abs(w)) <= 5000000

# Turn-Over Constraints

# Trading Constraints
trading_cost = np.array([0., 0., 0., 0., 0.])
# Holding Constraints
holding_cost = np.array([-1., -1., -1., -1., -1.])
# Market Impact Constraints
mi_cost = np.array([0., 0., 0., -0.1, -0.1])
mi_cost = (w - w_old) * mi_cost

gamma = Parameter(nonneg=True)
Lmax = Parameter()
GrossConstraints = Parameter()
risk = quad_form(f, vcv) + quad_form(w, np.array(specific_risk))

problem = Problem(Minimize(risk + norm(mi_cost, 1)),
                  [
                      w >= w_lb,
                      w <= w_ub,
                      # max(sum(w - w_old).value, sum(w_old - w).value) <= 2000000.,
                      #w <= w_ub,
                      # w[0] == 1000000.,
                      # [1] == -1000000.,
                      # w[2] == 2000000.,
                      # sum(abs(w - w_old).value) <= (Lmax * sum(abs(w_old))).value,
                      # f >= f_lb,
                      # f <= f_ub,
                      # sum(w) <= 4000000 # Net USD Constraints
                      # sum(abs(w).value) <= GrossConstraints.value
                     
                  ])

Lmax.value = .10
GrossConstraints.value = sum((w_old))

problem.solve(verbose=False)
print(" ** Checking Validity of Optimization ** ")
print("Problem Status:" + str(problem.status))
print("Optimized Risk:" + str(sqrt(risk).value))
print("Solution: " + str(w.value))



print("\n\n\n\nOptimization Summary")
print("Checking solutions with risk calculation ... ")
print("Old risk: " + str(calculate_portfolio_risk_factor(w_old, factor_exp, vcv, specific_risk)))
print("New risk: " + str(calculate_portfolio_risk_factor(w.value, factor_exp, vcv, specific_risk)))
print("Change In Position: " + str(w.value-w_old))
# print("Optimized Factor Value: " + str(f.value))
# print("Factor Constraints: " + str(f_lb))
# print("Factor Constraints: " + str(f_ub))
# print("Factor LB Satisfied: ", f.value >= f_lb)
# print("Factor UB Satisfied: ", f.value <= f_ub)
print("Gross Constraint Satisfied: ", (sum(abs(w)).value) <= GrossConstraints.value)
print("Gross Actual Value: " + str(sum(abs(w)).value / 1000000) + " vs Constraint: " + str(GrossConstraints.value / 1000000))
print("Turnover: " + str(sum(abs(w-w_old)).value / 1000000) + " vs Constraint: " + str(Lmax.value * sum(abs(w_old)).value / 1000000))


Current Portfolio Factor Exposure:[[-0.40525482 -0.69501158 -2.15829861]]
 ** Checking Validity of Optimization ** 
Problem Status:optimal
Optimized Risk:73672.23404706718
Solution: [1000000.00026084  999999.99596652 1999999.54915289  458894.45749089
  158735.96301375]




Optimization Summary
Checking solutions with risk calculation ... 
Old risk: 78420.37549681758
New risk: 73672.23404706718
Change In Position: [     0.00026084     -0.00403348     -0.45084711 458894.45749089
 158735.96301375]
Gross Constraint Satisfied:  False
Gross Actual Value: 4.617629965884885 vs Constraint: 4.0
Turnover: 0.6176308756460628 vs Constraint: 0.4


In [4]:
# Things to work on
# 1. Piecewise Market Impact Penalty
# 2. Drawing Plot using gridsearching for market impact, Trading, Holding and Risk Aversion Parameter

In [22]:
print(problem.status)

optimal


In [76]:
sum(w).value

3999983.3244577013

In [16]:
sum(w_old-w).value

-2620821.2214875673

In [ ]:
# # *************************************
# # Optimization - with Trading Cost 
# # n stocks
# # alp as ALPHA
# # n - alp as HEDGE
# # factor of 3 x 3
# # solve with w_new, w_old with $ positions
# np.random.seed(3)
# n = 5
# numFactor = 3
# dataseries = {}
# rowlist = []
# # Generate 500 price return series from 10 names
# for i, v in enumerate(range(numFactor)):
#     dataseries[v] = ((1 - np.random.lognormal(0, 0.03, numFactor)))
    
# for ke in dataseries.keys():
#     rowlist.append(dataseries[ke])

# # Generate Timeseries into pd
# timeseries = pd.DataFrame(rowlist)
# vcv = timeseries.cov()
# factor_exp = pd.DataFrame(np.random.randn(n, numFactor))
# specific_risk = pd.DataFrame(np.diag(np.random.uniform(0, 0.0020, n)))

# # set position vector
# w_old = [0.3, 0.3, 0.4, 0., 0.]

# # position constraints
# w_lb = [0.3, 0.3, 0.4, -1., -1.]
# w_ub = [1., 1., 1., 1., 1.]

# a1 = np.matmul(np.matrix(w_old), np.matrix(factor_exp))
# print("Current Portfolio Factor Exposure:" + str(a1 / 1000000))
# # # factor constraints
# # Current exp: -2.16, -4.11, -2.25
# #l = [x * 2 for x in l]
# f_lb = [1000000 * x for x in [-1., -1., -1.]]
# f_ub = [1000000 * x for x in [ 5.,  5.,  5.]]


# # # Set up the quadratic optimization problem
# w = Variable(n)
# f = np.array(factor_exp).T * w 

# # Gross Constraints
# # sum(abs(w)) <= 5000000

# # Turn-Over Constraints

# # Trading Constraints
# trading_cost = np.array([0., 0., 0., 0., 0.])
# # Holding Constraints
# holding_cost = np.array([-1., -1., -1., -1., -1.])
# # Market Impact Constraints
# mi_cost = np.array([0., 0., 0., -0.0010, -0.0030])
# mi_cost = abs(w - w_old) * mi_cost

# gamma = Parameter(nonneg=True)
# Lmax = Parameter()
# GrossConstraints = Parameter()
# risk = quad_form(f, vcv) + quad_form(w, np.array(specific_risk))

# problem = Problem(Minimize(risk + norm(mi_cost, 1)),
#                   [
#                       w >= w_lb,
#                       w <= w_ub,
#                       sum(w) <= 2.0,
#                       w[0] == 0.3
                      
#                       # max(sum(w - w_old).value, sum(w_old - w).value) <= 2000000.,
#                       #w <= w_ub,
#                       # w[0] == 1000000.,
#                       # [1] == -1000000.,
#                       # w[2] == 2000000.,
#                       # sum(abs(w - w_old).value) <= (Lmax * sum(abs(w_old))).value,
#                       #f >= f_lb,
#                       # f <= f_ub,
#                       # sum(abs(w).value) <= GrossConstraints.value
                     
#                   ])

# Lmax.value = .30
# GrossConstraints.value = sum((w_old))

# problem.solve(verbose=False)
# print(" ** Checking Validity of Optimization ** ")
# print("Problem Status:" + str(problem.status))
# print("Optimized Risk:" + str(sqrt(risk).value))
# print("Solution: " + str(w.value))



# print("\n\n\n\nOptimization Summary")
# print("Checking solutions with risk calculation ... ")
# print("Old risk: " + str(calculate_portfolio_risk_factor(w_old, factor_exp, vcv, specific_risk)))
# print("New risk: " + str(calculate_portfolio_risk_factor(w.value, factor_exp, vcv, specific_risk)))
# print("Change In Position: " + str(w.value-w_old))
# # print("Optimized Factor Value: " + str(f.value))
# # print("Factor Constraints: " + str(f_lb))
# # print("Factor Constraints: " + str(f_ub))
# # print("Factor LB Satisfied: ", f.value >= f_lb)
# # print("Factor UB Satisfied: ", f.value <= f_ub)
# print("Gross Constraint Satisfied: ", (sum(abs(w)).value) <= GrossConstraints.value)
# print("Gross Actual Value: " + str(sum(abs(w)).value) + " vs Constraint: " + str(GrossConstraints.value))
# print("Turnover: " + str(sum(abs(w-w_old)).value) + " vs Constraint: " + str(Lmax.value * sum(abs(w_old)).value))

In [20]:
# *************************************
# Optimization - with Trading Cost 
# n stocks
# alp as ALPHA
# n - alp as HEDGE
# factor of 3 x 3
# solve with w_new, w_old with $ positions
np.random.seed(3)
n = 5
numFactor = 3
dataseries = {}
rowlist = []
# Generate 500 price return series from 10 names
for i, v in enumerate(range(numFactor)):
    dataseries[v] = ((1 - np.random.lognormal(0, 0.03, numFactor)))
    
for ke in dataseries.keys():
    rowlist.append(dataseries[ke])

# Generate Timeseries into pd
timeseries = pd.DataFrame(rowlist)
vcv = timeseries.cov()
factor_exp = pd.DataFrame(np.random.randn(n, numFactor))
specific_risk = pd.DataFrame(np.diag(np.random.uniform(0, 0.0020, n)))

# set position vector
w_old = [1000000., 1000000., 2000000., 0., 0.]

# position constraints
w_lb = [999999.9, 999999.9, 1999999., -5000000., -5000000.]
w_ub = [1000000.1, 1000000.1, 2000000.1, 5000000., 5000000.]

a1 = np.matmul(np.matrix(w_old), np.matrix(factor_exp))
print("Current Portfolio Factor Exposure:" + str(a1 / 1000000))
# # factor constraints
# Current exp: -2.16, -4.11, -2.25
#l = [x * 2 for x in l]
f_lb = [1000000 * x for x in [-1., -1., -1.]]
f_ub = [1000000 * x for x in [ 5.,  5.,  5.]]


# # Set up the quadratic optimization problem
w = Variable(n)
f = np.array(factor_exp).T * w 
f_old = np.array(factor_exp).T * (w - w_old)

# Gross Constraints
# sum(abs(w)) <= 5000000


# Turn-Over Constraints

# Trading Constraints
trading_cost = np.array([0., 0., 0., -0.0030, -0.0020])
trading_cost = (w - w_old) * trading_cost

# Holding Constraints
holding_cost = np.array([-1., -1., -1., -1., -1.])

# Market Impact Constraints
# MI \proportionalto risk
# Problem Setup
# MI: http://faculty.haas.berkeley.edu/garleanu/DynTrad.pdf
# Dynamic Trading With Predictable Returns & Transaction Costs


gamma = Parameter(nonneg=True)
Lmax = Parameter(nonneg=True)
GrossConstraints = Parameter(nonneg=True)
Lambda_MI = Parameter(nonneg=True)
Lambda_TRDG = Parameter(nonneg=True)

risk = quad_form(f, vcv) + quad_form(w, np.array(specific_risk))
Market_Impact = quad_form(f - f_old, vcv) + quad_form(w - w_old, np.array(specific_risk))
# Linear_MI = cvxpy.sqrt(w.value)
Linear_MI = cvxpy.power(w.value, 1.5)


# problem = Problem(Minimize(risk + Lambda_MI * Market_Impact + Lambda_TRDG * trading_cost),
problem = Problem(Minimize(risk + Lambda_MI * Linear_MI + Lambda_TRDG * trading_cost),
                  [
                      w >= w_lb,
                      w <= w_ub,
                      # max(sum(w - w_old).value, sum(w_old - w).value) <= 2000000.,
                      #w <= w_ub,
                      # w[0] == 1000000.,
                      # [1] == -1000000.,
                      # w[2] == 2000000.,
                      # sum(abs(w - w_old).value) <= (Lmax * sum(abs(w_old))).value, # Turnover Constraints
                      # f >= f_lb,
                      # f <= f_ub,
                      # sum(w) <= 4000000 # Net USD Constraints 
                      # sum(abs(w).value) <= GrossConstraints.value # Gross Constraints
                     
                  ])

Lmax.value = .10
GrossConstraints.value = sum((w_old))
Lambda_MI.value = 0.5
Lambda_TRDG.value = 1.

problem.solve(verbose=False)
print(" ** Checking Validity of Optimization ** ")
print("Problem Status:" + str(problem.status))
print("Optimized Risk:" + str(sqrt(risk).value))
print("Solution: " + str(w.value))



print("\n\n\n\nOptimization Summary")
print("Checking solutions with risk calculation ... ")
print("Old risk: " + str(calculate_portfolio_risk_factor(w_old, factor_exp, vcv, specific_risk)))
print("New risk: " + str(calculate_portfolio_risk_factor(w.value, factor_exp, vcv, specific_risk)))
print("Change In Position: " + str(w.value-w_old))
# print("Optimized Factor Value: " + str(f.value))
# print("Factor Constraints: " + str(f_lb))
# print("Factor Constraints: " + str(f_ub))
# print("Factor LB Satisfied: ", f.value >= f_lb)
# print("Factor UB Satisfied: ", f.value <= f_ub)
print("Gross Constraint Satisfied: ", (sum(abs(w)).value) <= GrossConstraints.value)
print("Gross Actual Value: " + str(sum(abs(w)).value / 1000000) + " vs Constraint: " + str(GrossConstraints.value / 1000000))
print("Turnover: " + str(sum(abs(w-w_old)).value / 1000000) + " vs Constraint: " + str(Lmax.value * sum(abs(w_old)).value / 1000000))



Current Portfolio Factor Exposure:[[-0.40525482 -0.69501158 -2.15829861]]
 ** Checking Validity of Optimization ** 
Problem Status:optimal
Optimized Risk:73672.21413871077
Solution: [ 999999.98568569  999999.99136004 1999999.00284219  458894.70390355
  158736.30861889]




Optimization Summary
Checking solutions with risk calculation ... 
Old risk: 78420.37549681758
New risk: 73672.21413871077
Change In Position: [    -0.01431431     -0.00863996     -0.99715781 458894.70390355
 158736.30861889]
Gross Constraint Satisfied:  False
Gross Actual Value: 4.6176299924103485 vs Constraint: 4.0
Turnover: 0.6176320326345289 vs Constraint: 0.4


In [86]:
f_old.shape

(3, 5)

In [87]:
f.shape

(3,)

In [88]:
f

Expression(AFFINE, UNKNOWN, (3,))

In [100]:
np.array(factor_exp).T * w.value

array([[ -477218.03606426,   881318.05171678,  -809354.65684042,
          300535.25016316,   -21762.68481644],
       [-1313864.76906889,  1709573.0820989 , -1090719.66295779,
         -336849.15045255,   157270.09024941],
       [  884622.39107454,    50033.64271671, -3092953.97248588,
         -362540.77856514,    25050.25059117]])

In [103]:
mi_cost.value
trading_cost = np.array([0., 0., 0., 5., 5.])
#trading_cost = (w - w_old) * trading_cost

In [107]:
((w-w_old) * trading_cost).value

2058767.6208629627

In [5]:
cvxpy.sqrt(5)

Expression(CONSTANT, NONNEGATIVE, ())